## Hangman Algorithm

In [274]:
# Load modules 
import re
import operator
import json
import math
from collections import Counter

In [147]:
# Load english word dictonary 
def open_and_clean_english_dict(text_file = 'words.txt'):
    with open(text_file) as f:
        english_dict = f.read().splitlines()
    english_dict = list(set([x.lower() for x in english_dict]))
    english_dict = [re.sub(r'[^\w\s]','',x) for x in english_dict] 
    return english_dict 

In [148]:
def reset_game_metrics():
    guesses_list = []
    num_guesses = 0
    num_misses = 0
    correct_guess_dict = {}
    game_over = False 
    return guesses_list, num_guesses, num_misses, correct_guess_dict, game_over

In [149]:
def pre_process_input(inputword):
    outputword = str(inputword).lower()
    return outputword

In [150]:
## more sophisticated might weight distribution by actual use/commonly known words 
def get_words_with_len(word_length):
    start_words = [word for word in english_dict if len(word) == word_length]
    return start_words

In [151]:
def get_letter_dist_from_word_list(list_of_words):
    all_letters = [list(w) for w in list_of_words]
    all_letters_flat = [item for sublist in all_letters for item in sublist]
    letter_count = Counter(all_letters_flat)
    return letter_count

In [152]:
def get_word_so_far(correct_guess_dict):
    word_so_far = []
    for i in range(0,input_word_length):
        if i in correct_guess_dict.keys():
            word_so_far.append(correct_guess_dict[i])
        else:
            word_so_far.append("_")
    word_so_far = str(word_so_far)
    return word_so_far

In [153]:
## SEARCH BY DISTRIBUTION TO FIND FIRST LETTER 
def guess_using_dist(dist):
    global guesses_list
    global num_misses
    global num_guesses
    global correct_guess_dict
    global dists_used 
    
    dist = [x for x in ordered_letter_dist if x[0] not in guesses_list]  # remove guesses already made
    letter_freq_tuple = dist[0] # get the largest letter frequency  
    letter = letter_freq_tuple[0] # get letter only 
    print("LETTER GUESSED: ", letter)
    guesses_list.append(letter)
    num_guesses += 1
    match_found = False 
    for index, position in enumerate(inputword):
        if letter == position:
            correct_guess_dict[index] = letter
            match_found = True 
            print("MATCH FOUND")
            print("NUMBER OF MISSES", num_misses)
            print("WORD SO FAR", get_word_so_far(correct_guess_dict))
            print(" ")
    if match_found == False:
        print("NO MATCH FOUND")
        num_misses += 1
        print("NUMBER OF MISSES:", num_misses)
        print("WORD SO FAR", get_word_so_far(correct_guess_dict))
        print(" ")
    return correct_guess_dict, guesses_list, num_guesses, num_misses 

In [161]:
## RECALC DISTRIBUTION WITH NEW WORD INFO 
def get_remaining_words(word_list_in, correct_guess_dict,guesses_list):

    remain_words = word_list_in 
    
    # remove words with letters guessed which are wrong and therefore not in it 
    wrong_letters = [letter for letter in guesses_list if letter not in list(correct_guess_dict.values())]
    remain_words_out1 = remain_words 
    for guess_letter in wrong_letters:
        remain_words_out1 = [word for word in remain_words if guess_letter not in word]

    # keep only words with correctly guessed letters in the same position 
    remain_words_out2 = remain_words_out1
    for i in correct_guess_dict.keys():
        correct_letter = correct_guess_dict[i]
        remain_words_out2 = [word for word in remain_words_out1 if correct_letter == word[i]]
        
    print("NUMBER OF WORDS REMAINING", len(remain_words_out2))
    if len(remain_words_out2) < 11:
        print(remain_words_out2)
    return remain_words_out2


## Play game

In [162]:
guesses_list, num_guesses, num_misses, correct_guess_dict, game_over = reset_game_metrics()
english_dict = open_and_clean_english_dict()

In [163]:
## input word 
inputword = 'snip'
#inputword = input()

if len([x for x in english_dict if inputword in x]) == 0:
    print("Word is not in the dictonary, try another")
else:
    print("Word is in the dictonary, execute below to play!")

Word is in the dictonary, execute below to play!


In [164]:
inputword = pre_process_input(inputword)
input_word_length = len(inputword)
start_words = get_words_with_len(input_word_length)

while len(correct_guess_dict.keys()) < input_word_length:
    letter_dist = get_letter_dist_from_word_list(start_words)
    ordered_letter_dist = letter_dist.most_common()

    correct_guess_dict, guesses_list, num_guesses, num_misses = guess_using_dist(ordered_letter_dist)
    start_words = get_remaining_words(start_words, correct_guess_dict,guesses_list)
else:
    final_word = str([correct_guess_dict[x] for x in range(0,len(correct_guess_dict.keys()))])
    print("GAME OVER", final_word)
    print("FINAL NUMBER OF MISSES", num_misses)
    guesses_list, num_guesses, num_misses, correct_guess_dict, game_over = reset_game_metrics()

GAME OVER ['s', 'n', 'i', 'p']
FINAL NUMBER OF MISSES 5


In [165]:
## TODO

# double same letter bug [X]
# find better dictonary with plurals etc.  [x]
# change finish condition to having all letters not only one candidate left [x]
# clean dictonary of punctuation [x]
# could it be improved by guesses when small number of words left even at cost of a miss? 
    # - unlikely as probabiltiy of randomly right word = 1/num_of_remaining_words, 
    # - which would need to be larger than 1/26-num_guesses at best - even before weighted distribution is considered 
# remove stopwords?
# weight words by commonality 


## Backcalculate toughest word 

In [286]:
words_to_search = english_dict[0:204]

In [283]:
%%time

number_of_batches = 4

batch_size = len(words_to_search) / number_of_batches
batch_size = math.ceil(batch_size)

results = {}
batch_number = 0
batch_amount = 0
run_size = 0

for search_word in words_to_search:
    inputword = search_word
    inputword = pre_process_input(inputword)
    input_word_length = len(inputword)
    start_words = get_words_with_len(input_word_length)

    while len(correct_guess_dict.keys()) < input_word_length:
        letter_dist = get_letter_dist_from_word_list(start_words)
        ordered_letter_dist = letter_dist.most_common()

        correct_guess_dict, guesses_list, num_guesses, num_misses = guess_using_dist(ordered_letter_dist)
        start_words = get_remaining_words(start_words, correct_guess_dict,guesses_list)
    else:
        final_word = str([correct_guess_dict[x] for x in range(0,len(correct_guess_dict.keys()))])

        results[search_word] = num_misses

        guesses_list, num_guesses, num_misses, correct_guess_dict, game_over = reset_game_metrics()
        
        batch_amount += 1
        run_size += 1
        if batch_amount == batch_size:
            results_file = str('results{}'.format(str(batch_number))) + '.json'
            print("batch number",str(batch_number))
            with open(results_file, 'a') as fp:
                json.dump(results, fp)
            print("records complete... ",str(run_size))
            batch_amount = 0
            batch_number += 1
            results = {}

# to flush the final batch 
results_file = str('results{}'.format(str(batch_number))) + '.json'
print("batch number",str(batch_number))
with open(results_file, 'a') as fp:
    json.dump(results, fp)
print("records complete... ",str(run_size))

batch number 0
records complete...  51
batch number 1
records complete...  102
batch number 2
records complete...  153
batch number 3
records complete...  204
batch number 4
records complete...  204
Wall time: 34.2 s


## Reading results

In [253]:
# def Merge(dict1, dict2): 
#     res = {**dict1, **dict2} 
#     return res 

# main_dict = {}

# for x in range(0,2):
#     file_name = str('results{}'.format(str(x))) + '.json'
#     with open(file_name) as data_file:    
#         data = json.load(data_file)
#     main_dict = Merge(main_dict,data)

In [275]:
# import pandas as pd
# df = pd.Series(main_dict, name='num_misses').reset_index()

In [ ]:
# toughest_word = max(results.items(), key=operator.itemgetter(1))[0]
# largest_num_misses = results[toughest_word]
# toughest_words = [k for k,v in results.items() if v == largest_num_misses]